In [ ]:
!pip install -q transformers datasets evaluate wandb

In [ ]:
from datasets import load_dataset

from evaluate import evaluator

from huggingface_hub import notebook_login

from sklearn.metrics import accuracy_score, f1_score

import torch

from torch.nn import CrossEntropyLoss

from torch.optim import AdamW

from transformers import AutoTokenizer, AutoModel, PreTrainedModel
from transformers import Trainer, TrainingArguments

from transformers import RobertaConfig

from transformers.optimization import get_linear_schedule_with_warmup
from transformers.modeling_outputs import SequenceClassifierOutput
import wandb

In [ ]:
%env WANDB_PROJECT=emotion_classifier

In [ ]:
wandb.login()

In [ ]:
notebook_login()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
def tokenization(sample):
    return tokenizer(sample["text"], padding=True, truncation=True)

## Dataset

In [ ]:
dataset = load_dataset("emotion")

In [ ]:
dataset = dataset.map(tokenization, batched=True, batch_size=None)

In [ ]:
dataset.set_format("torch", columns=["input_ids", "label"])

In [ ]:
id2label =  {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
  }

label2id = {
    "sadness": 0,
    "joy": 1,
    "love": 2,
    "anger": 3,
    "fear": 4,
    "surprise": 5
  }

## Model

In [ ]:
config = RobertaConfig.from_pretrained("roberta-base", 
                                       id2label = id2label, 
                                       label2id = label2id, 
                                       num_labels = 6)

In [ ]:
class RobertaEmotion(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.backbone = AutoModel.from_pretrained("roberta-base", config)
        self.dropout = torch.nn.Dropout(p=0.1)
        self.output = torch.nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, labels=None, attention_mask=None):
        model_output = self.backbone(input_ids)
        hidden = model_output.last_hidden_state
        logits = self.output(self.dropout(hidden[:, 0, :]))

        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(loss=loss, logits=logits)

In [ ]:
model = RobertaEmotion(config).to(device)

## Training

In [ ]:
model.base_model.requires_grad = False

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
batch_size = 64
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=8,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  report_to="wandb",
                                  disable_tqdm=False,
                                  push_to_hub=False,
                                  hub_strategy="every_save",
                                  hub_model_id="ma2za/roberta-emotion")

In [ ]:
optimizer = AdamW(model.parameters(),lr= 2e-05, betas= (0.9, 0.999), eps= 1e-08)


lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=2000)


trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=dataset["train"],
                  eval_dataset=dataset["validation"], 
                  optimizers=[optimizer, lr_scheduler])
trainer.train()

wandb.finish()

In [ ]:
model.push_to_hub("roberta-emotion")
tokenizer.push_to_hub("roberta-emotion")

## Evaluation

In [ ]:
task_evaluator = evaluator("text-classification")

In [ ]:
results = task_evaluator.compute(
    model_or_pipeline=model,
    tokenizer=tokenizer,
    data="emotion",
    subset="split",
    split="validation",
    metric="accuracy",
    label_mapping=label2id,
    strategy="bootstrap",
    n_resamples=10,
    random_state=0
)

In [ ]:
results